In [5]:
import os 
import pandas as pd
import path
import pickle 
import numpy as np
import h5py

In [6]:
base_path_junctions = '/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes'

In [136]:
folder_OV_jx_unfiltered = 'TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv'
folder_OV_jx_conf2 = 'TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv'

folfer_BRCA_jx_unfiltered = ''

count_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.rechunked.hdf5')
#count_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.hdf5')
splice_path_OV = os.path.join( base_path_junctions, 'TCGA_Ovarian_374_results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle')
sample_file_OV = '/cluster/work/grlab/projects/projects2020_OHSU/sample_lists/TCGA_foreground/sample_full_Ov_378.tsv'

count_path_BRCA = os.path.join( base_path_junctions, 'TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.count.hdf5')
splice_path_BRCA = os.path.join( base_path_junctions, 'TCGA_Breast_1102_results/splicing/spladder/genes_graph_conf2.merge_graphs.pickle')
sample_file_BRCA = "/cluster/work/grlab/projects/projects2020_OHSU/sample_lists/TCGA_foreground/sample_full_BRCA_1102.tsv"

In [ ]:
sample = 'sample1' # relink samples 1 to 5

In [ ]:
path_OV_jx_unfiltered = os.path.join(base_path_junctions, folder_OV_jx_unfiltered, sample)
print(path_OV_jx_unfiltered)
path_OV_jx_conf2 = os.path.join(base_path_junctions, folder_OV_jx_conf2, sample)
print(path_OV_jx_conf2)
path_OV_jx_unfiltered = os.path.realpath(path_OV_jx_unfiltered)
print(path_OV_jx_unfiltered)
path_OV_jx_conf2 = os.path.realpath(path_OV_jx_conf2)
print(path_OV_jx_conf2)

In [26]:
sample_name = path_OV_jx_unfiltered.split('/')[-1].replace('.junctions.tsv.gz', '')

# "Spladder files"

In [23]:
# Load
df_OV_jx_unfiltered = pd.read_csv(path_OV_jx_unfiltered,
                                  sep = '\t')
df_OV_jx_conf2 = pd.read_csv(path_OV_jx_conf2, 
                                  sep = '\t')

/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv/sample1
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv/sample1
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv/TCGA-25-1319-01A-01R-1565-13.all.junctions.tsv.gz
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv/TCGA-25-1319-01A-01R-1565-13.all.conf_2.filt.junctions.tsv.gz


'TCGA-25-1319-01A-01R-1565-13.all'

In [66]:
def unique_junctions_df(df):
    display(df.head(3))
    print('shape table')
    print(df.shape)
    print('shape table excluding column count')
    print(df.drop(['count'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, chr')
    print(df.drop(['count', 'chr'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, strand')
    print(df.drop(['count', 'strand'], axis = 1).drop_duplicates().shape)
    print('shape table excluding column count, strand, chr')
    print(df.drop(['count', 'strand', 'chr'], axis = 1).drop_duplicates().shape)
    

In [67]:
print(path_OV_jx_unfiltered)
unique_junctions_df(df_OV_jx_unfiltered)
print(path_OV_jx_conf2)
unique_junctions_df(df_OV_jx_conf2)

/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_unfiltered_tsv/TCGA-25-1319-01A-01R-1565-13.all.junctions.tsv.gz


,chr,strand,junction_start,junction_end,count
0,chr10,-,48114,48614,1
1,chr10,-,48725,48803,3
2,chr10,-,277577,281199,12


shape table
(485934, 5)
shape table excluding column count
(485934, 4)
shape table excluding column count, chr
(485747, 3)
shape table excluding column count, strand
(485436, 3)
shape table excluding column count, strand, chr
(485249, 2)
/cluster/work/grlab/projects/projects2021-immuno_peptides/results/TCGA_for_neoepitopes/TCGA_Ovarian_374_results/TCGA_Ovarian_374_results_junctions_conf_2_tsv/TCGA-25-1319-01A-01R-1565-13.all.conf_2.filt.junctions.tsv.gz


,chr,strand,junction_start,junction_end,count
0,chr10,-,48114,48614,1
1,chr10,-,48725,48803,1
2,chr10,-,277577,281199,9


shape table
(396650, 5)
shape table excluding column count
(396650, 4)
shape table excluding column count, chr
(396510, 3)
shape table excluding column count, strand
(396405, 3)
shape table excluding column count, strand, chr
(396265, 2)


# Graph Single Gene

In [ ]:
graph, meta = pickle.load(open(splice_path_OV, 'rb'))

In [38]:
gene = graph[1]
gene.from_sparse()

In [107]:
gene.name

'ENSG00000227232.5'

In [103]:
gene.segmentgraph.seg_edges.shape

(103, 103)

In [106]:
gene.splicegraph.edges.shape

(100, 100)

In [39]:
#vars(gene.splicegraph)

In [40]:
#[(a,b) for a, b in zip(np.where(gene.segmentgraph.seg_edges)[0],np.where(gene.segmentgraph.seg_edges)[1])]

In [119]:
gene.splicegraph.edges[0:20, 0:20]


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [122]:
edges_flattened = gene.splicegraph.edges.flatten()

In [116]:
edges_graph_index = np.array([(a,b) for a, b in zip(np.where(gene.splicegraph.edges)[0],
                                               np.where(gene.splicegraph.edges)[1])])
print(np.array(edges_graph_index)[0:10])
print(np.array(edges_graph_index).shape)

[[ 0  9]
 [ 0 10]
 [ 0 11]
 [ 1 10]
 [ 1 46]
 [ 2  9]
 [ 2 10]
 [ 2 17]
 [ 3 10]
 [ 3 26]]
(478, 2)


In [127]:
edges_graph_index_flattened = np.where(edges_flattened)[0]
print(edges_graph_index_flattened)

[   9   10   11  110  146  209  210  217  310  326  327  410  435  510
  547  608  609  610  612  617  618  619  620  625  635  709  710  717
  806  817  818  820  900  902  906  907  917  918  919  920  925 1000
 1001 1002 1003 1004 1005 1006 1007 1014 1015 1016 1017 1018 1019 1020
 1021 1025 1035 1100 1117 1206 1217 1317 1410 1437 1510 1535 1610 1632
 1635 1702 1706 1707 1708 1709 1710 1711 1712 1713 1730 1731 1732 1733
 1734 1735 1736 1806 1808 1809 1810 1835 1836 1846 1906 1909 1910 1935
 2006 2008 2009 2010 2035 2036 2046 2110 2146 2235 2236 2335 2435 2506
 2509 2510 2535 2603 2631 2635 2703 2731 2735 2835 2932 2934 2935 2936
 3017 3038 3117 3126 3127 3139 3216 3217 3229 3240 3242 3317 3346 3417
 3429 3441 3504 3506 3510 3515 3516 3517 3518 3519 3520 3522 3523 3524
 3525 3526 3527 3528 3529 3543 3544 3545 3546 3547 3617 3618 3620 3622
 3629 3646 3647 3714 3746 3830 3846 3931 3946 4032 4046 4134 4143 4146
 4232 4246 4335 4341 4350 4351 4435 4448 4535 4549 4601 4618 4620 4621
 4633 

In [115]:
# CHECK ORDER
junction_gene = [(gene.splicegraph.vertices[0][i], gene.splicegraph.vertices[1][j]) for i, j in edges_graph_index]

print(junction_gene[0:10])

[(14403, 15038), (14403, 15038), (14403, 15059), (14403, 15038), (14403, 17055), (14403, 15038), (14403, 15038), (14403, 15947), (14403, 15038), (14403, 16027)]


In [82]:
len(junction_gene)

478

In [63]:
df_junction_gene = pd.DataFrame(junction_gene, columns = ['junction_start', 'junction_end'])

# Count File

In [148]:
# Count file 
h5f = h5py.File(count_path_OV, 'r')
print(h5f.keys())
for k in h5f.keys():
    print(k)
    print(h5f[k].shape)
    sample_id = [i for i, j in enumerate(h5f['strains']) if j.decode() == sample_name][0]
    edge_expr_sample = h5f['edges'][:, sample_id]


<KeysViewHDF5 ['edge_idx', 'edges', 'gene_ids_edges', 'gene_ids_segs', 'gene_names', 'seg_len', 'seg_pos', 'segments', 'strains']>
edge_idx
(1979609,)
edges
(1979609, 374)
gene_ids_edges
(1979609, 1)
gene_ids_segs
(2243686, 1)
gene_names
(60609, 1)
seg_len
(2243686, 1)
seg_pos
(2243686, 374)
segments
(2243686, 374)
strains
(374,)


In [149]:
gene_hd5_id = [i for i, j in enumerate(h5f['gene_names'][:, 0]) if j.decode() == gene.name][0]
print('gene id is:', gene_hd5_id)

gene id is: 1


In [150]:
edges_index_gene = [i for i, j in enumerate(h5f['gene_ids_edges'][:, 0]) if j == gene_hd5_id]
print('found', len(edges_index_gene), 'edges corresponding to the gene')
print(edges_index_gene)


found 122 edges corresponding to the gene
[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137]


In [151]:
segm_index_gene = [i for i, j in enumerate(h5f['gene_ids_segs'][:, 0]) if j == gene_hd5_id]
print('found', len(edges_index_gene), 'segments corresponding to the gene')
print(segm_index_gene)


found 122 segments corresponding to the gene
[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121]


In [ ]:
#https://ratschlab.slack.com/archives/DD4F40KPF/p1648498023344119

In [140]:
edges_index_gene_struct = h5f['edge_idx'][edges_index_gene] # WHAT ARE THOSE???
print('found the following edge idx...', edges_index_gene_struct)
edges_index_gene_struct = [int(i) for i in edges_index_gene_struct]
print('found the following edge idx...', edges_index_gene_struct)

found the following edge idx... [9.000e+00 1.000e+01 1.130e+02 1.460e+02 2.150e+02 2.160e+02 2.200e+02
 3.190e+02 3.280e+02 3.290e+02 4.220e+02 4.410e+02 5.250e+02 5.590e+02
 6.260e+02 6.270e+02 6.280e+02 6.290e+02 6.320e+02 6.360e+02 6.470e+02
 7.300e+02 7.310e+02 7.350e+02 1.147e+03 1.151e+03 1.162e+03 1.250e+03
 1.353e+03 1.576e+03 1.677e+03 2.089e+03 2.192e+03 2.295e+03 2.309e+03
 2.398e+03 2.707e+03 2.810e+03 2.824e+03 3.021e+03 3.230e+03 3.334e+03
 3.336e+03 3.545e+03 3.644e+03 4.162e+03 4.163e+03 4.164e+03 4.266e+03
 4.267e+03 4.579e+03 4.683e+03 4.993e+03 5.098e+03 5.306e+03 5.514e+03
 5.617e+03 5.653e+03 5.721e+03 5.729e+03 5.734e+03 5.748e+03 5.756e+03
 5.761e+03 5.762e+03 5.763e+03 5.837e+03 5.859e+03 6.065e+03 6.146e+03
 6.168e+03 6.244e+03 6.249e+03 6.260e+03 6.263e+03 6.271e+03 6.276e+03
 6.277e+03 6.280e+03 6.347e+03 6.352e+03 6.363e+03 6.366e+03 6.374e+03
 6.379e+03 6.380e+03 6.383e+03 6.450e+03 6.455e+03 6.469e+03 6.477e+03
 6.482e+03 6.483e+03 6.661e+03 6.675e+03 6.68

In [141]:
set(edges_graph_index_flattened).intersection(set(edges_index_gene_struct))

{9, 10, 146, 3545, 5653, 5756, 6168, 6276, 6277, 6374, 6661}

In [146]:
edges_graph_index_flattened

array([   9,   10,   11,  110,  146,  209,  210,  217,  310,  326,  327,
        410,  435,  510,  547,  608,  609,  610,  612,  617,  618,  619,
        620,  625,  635,  709,  710,  717,  806,  817,  818,  820,  900,
        902,  906,  907,  917,  918,  919,  920,  925, 1000, 1001, 1002,
       1003, 1004, 1005, 1006, 1007, 1014, 1015, 1016, 1017, 1018, 1019,
       1020, 1021, 1025, 1035, 1100, 1117, 1206, 1217, 1317, 1410, 1437,
       1510, 1535, 1610, 1632, 1635, 1702, 1706, 1707, 1708, 1709, 1710,
       1711, 1712, 1713, 1730, 1731, 1732, 1733, 1734, 1735, 1736, 1806,
       1808, 1809, 1810, 1835, 1836, 1846, 1906, 1909, 1910, 1935, 2006,
       2008, 2009, 2010, 2035, 2036, 2046, 2110, 2146, 2235, 2236, 2335,
       2435, 2506, 2509, 2510, 2535, 2603, 2631, 2635, 2703, 2731, 2735,
       2835, 2932, 2934, 2935, 2936, 3017, 3038, 3117, 3126, 3127, 3139,
       3216, 3217, 3229, 3240, 3242, 3317, 3346, 3417, 3429, 3441, 3504,
       3506, 3510, 3515, 3516, 3517, 3518, 3519, 35

In [ ]:
# WHY ARE THE INDEXE NOT MATCHING??? PROBABLY BECAUSE OF TH DIGONAL INDEXING//
#edges_index_gene_struct should be a subset of edges_graph_index_flattened

In [144]:
edge_expr_sample_gene = edge_expr_sample[edges_index_gene]
print('found following edge expression with first set of IDS \n ', edge_expr_sample_gene)
print ('amoung which non-null', sum(edge_expr_sample_gene > 0))

edge_expr_sample_gene = edge_expr_sample[edges_index_gene_struct]
print('found following edge expression with first set of IDS \n ', edge_expr_sample_gene)
print ('amoung which non-null', sum(edge_expr_sample_gene > 0))


found following edge expression with first set of IDS 
  [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. 104.   0.   0.   0.   0.   0.   0.   0.   0.  15.   0.   0.   0.
   0.   0.   0.   0.  19.   1.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   1.  58.   0.   0.   0.   0.   0. 204. 106.   0.   0.
 104.   0. 115.   0.   0.   0.   4.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.  74.   3.   0.   0.   2.   5.   0.   0.   5.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   4.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.  61.   3.   0.   0.   0.   0.]
amoung which non-null 19
found following edge expression with first set of IDS 
  [  0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   2.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   

In [ ]:
# FIGURE OUT WHICH OF THE TWO INDEXINGS ABOVE IS CORRECT
# GET BACK THE IDS THAT ARE NOT NULL 
# GET THEM BACK FROM THE GRAPH

In [145]:
# CLOSE
h5f.close()

# Graph

In [37]:
graph, meta = pickle.load(open(splice_path_OV, 'rb'))

In [86]:
junction_all_genes = []
for idx, gene in enumerate(graph):
    if idx % 1000 == 0:
        print(idx)
    gene.from_sparse()
    edges_coords = np.array([(a,b) for a, b in zip(np.where(gene.splicegraph.edges)[0],np.where(gene.splicegraph.edges)[1])])
    junction_gene = [(gene.splicegraph.vertices[0][i], gene.splicegraph.vertices[1][j]) for i, j in edges_coords]
    junction_all_genes.extend(junction_gene)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000


In [101]:
df_junction_gene = pd.DataFrame(junction_all_genes, columns = ['junction_start', 'junction_end'])


In [102]:
df_junction_gene.head()

,junction_start,junction_end
0,11011,12057
1,11471,12057
2,11644,12057
3,11868,12696
4,11868,12697


In [103]:
df_junction_gene.shape

(21054558, 2)

In [104]:
df_junction_gene.drop_duplicates().shape

(7270711, 2)

In [105]:
jxS_graph_unfilt = df_junction_gene.drop_duplicates().merge(\
    df_OV_jx_unfiltered.loc[:, ['junction_start', 'junction_end']].drop_duplicates(), 
                                          on = ['junction_start', 'junction_end'], how = 'inner')

In [106]:
jxS_graph_unfilt.drop_duplicates().shape

(561, 2)

# TEST SINGLE GENE MERGE

In [75]:
junction_graph_unfilt = df_junction_gene.merge(df_OV_jx_unfiltered, on = ['junction_start', 'junction_end'], how = 'left')

In [80]:
display(junction_graph_unfilt.head())
print(junction_graph_unfilt.shape)
print(junction_graph_unfilt.drop_duplicates().shape)
junction_graph_unfilt = junction_graph_unfilt.merge(df_OV_jx_unfiltered, on = ['junction_start', 'junction_end'], how = 'left')

,junction_start,junction_end,chr,strand,count
0,14403,15038,NaN,NaN,NaN
1,14403,15038,NaN,NaN,NaN
2,14403,15059,NaN,NaN,NaN
3,14403,15038,NaN,NaN,NaN
4,14403,17055,NaN,NaN,NaN


(478, 5)
(254, 5)


In [81]:
junction_graph_unfilt.head()

,junction_start,junction_end,chr_x,strand_x,count_x,chr_y,strand_y,count_y
0,14403,15038,NaN,NaN,NaN,NaN,NaN,NaN
1,14403,15038,NaN,NaN,NaN,NaN,NaN,NaN
2,14403,15059,NaN,NaN,NaN,NaN,NaN,NaN
3,14403,15038,NaN,NaN,NaN,NaN,NaN,NaN
4,14403,17055,NaN,NaN,NaN,NaN,NaN,NaN
